<ul class="breadcrumb">
  <li><a href="1_Conventional_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Conventional)</a></li>
  <li><a href="2_Linear_Additive_Genetic_Model.ipynb">Bayesian Linear Additive Genetic Model</a></li> 
  <li><a href="3_Genomic_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Genomic Data)</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Single-step Bayesian Regression (Incomplete Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 1: Load Packages</button> 

In [48]:
using JWAS,JWAS.Datasets,DataFrames,CSV, LinearAlgebra

<button type="button" class="btn btn-lg btn-primary">Step 2: Read data</button> 

In [49]:
phenofile  = Datasets.dataset("example","phenotypes_ssbr.txt")
pedfile    = Datasets.dataset("example","pedigree.txt")
genofile   = Datasets.dataset("example","genotypes.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true,missingstrings=["NA"])
pedigree   = get_pedigree(pedfile,separator=",",header=true);

The delimiter in pedigree.txt is ','.
Pedigree informatin:
#individuals: 12
#sires:       4
#dams:        5
#founders:    3


In [51]:
first(phenotypes,5)

,ID,y1,y2,y3,x1,x2,x3,dam,weights
,String,Float64,Float64,Float64,Float64,Int64,String,String⍰,Float64
1,a1,-0.06,3.58,-1.18,0.9,2,m,missing,1.0
2,a2,-0.6,4.9,0.88,0.3,1,f,missing,1.0
3,a3,-2.07,3.19,0.73,0.7,2,f,missing,1.0
4,a4,-2.63,6.97,-0.83,0.6,1,m,a2,1.0
5,a5,2.31,3.5,-1.52,0.4,2,m,a2,1.0


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Single-trait Single-step Bayesian Regression (Incomplete Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [52]:
model_equation1  ="y1 = intercept + x1*x3 + x2 + x3 + ID + dam";

In [53]:
model1 = build_model(model_equation1);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [54]:
set_covariate(model1,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [55]:
set_random(model1,"x2");
set_random(model1,"ID dam",pedigree);

<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [56]:
add_genotypes(model1,genofile,separator=',');

The delimiter in genotypes.txt is ','.
The header (marker IDs) is provided in genotypes.txt.
5 markers on 7 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [57]:
out1=runMCMC(model1,phenotypes,methods="RR-BLUP",single_step_analysis=true,pedigree=pedigree);

Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
The number of observations with both phenotype and pedigree information used in the analysis is 8.
Prior information for genomic variance is not provided and is generated from the data.
Prior information for residual variance is not provided and is generated from the data.
Prior information for random effect variance is not provided and is generated from the data.
Prior information for random effect variance is not provided and is generated from the data.
calculating A inverse
  0.000082 seconds (205 allocations: 16.031 KiB)
imputing missing genotypes
  0.195236 seconds (190 allocations: 23.781 KiB, 99.87% gc time)
completed imputing genotypes
Missing values are found in independent variables: dam.

The prior for marker effects variance is calculated from the genetic variance and π.
The mean of the prior for the marker effects variance is: 0.496268



A Linear Mixed Model was build 

<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [58]:
keys(out1)

Base.KeySet for a Dict{Any,Any} with 6 entries. Keys:
  "marker effects"
  "EBV_y1"
  "location parameters"
  "residual variance"
  "polygenic effects covariance matrix"
  "marker effects variance"

In [59]:
out1["EBV_y1"]

,ID,EBV,PEV
,Any,Any,Any
1,a1,0.308266,6.46324
2,a3,-0.992717,6.05282
3,a4,-0.399613,7.90331
4,a5,0.671401,4.17104
5,a6,0.0388337,7.78589
6,a7,-1.41756,6.00083
7,a8,-0.166578,8.37988


In [60]:
out1["marker effects"]

,Trait,Marker_ID,Estimate,Std_Error,Model_Frequency
,Any,Any,Any,Any,Any
1,y1,m1,-0.112787,0.535923,1.0
2,y1,m2,-0.256412,0.670566,1.0
3,y1,m3,0.272158,0.607386,1.0
4,y1,m4,-0.275205,0.563826,1.0
5,y1,m5,-0.00133083,0.535076,1.0


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multi-trait Single-step Bayesian Regression (Incomplete Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [61]:
model_equation2 ="y1 = intercept + x1 + x3 + ID + dam
                  y2 = intercept + x1 + x2 + x3 + ID
                  y3 = intercept + x1 + x1*x3 + x2 + ID";

In [62]:
model2 = build_model(model_equation2);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [63]:
set_covariate(model2,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [64]:
set_random(model2,"x2");
set_random(model2,"ID dam",pedigree);

x2 is not found in model equation 1.
dam is not found in model equation 2.
dam is not found in model equation 3.


<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [65]:
add_genotypes(model2,genofile,separator=',');

The delimiter in genotypes.txt is ','.
The header (marker IDs) is provided in genotypes.txt.
5 markers on 7 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [66]:
out2=runMCMC(model2,phenotypes,methods="BayesC",estimatePi=true,single_step_analysis=true,pedigree=pedigree);

Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
The number of observations with both phenotype and pedigree information used in the analysis is 8.
Prior information for genomic variance is not provided and is generated from the data.
Prior information for residual variance is not provided and is generated from the data.
Prior information for random effect variance is not provided and is generated from the data.
Prior information for random effect variance is not provided and is generated from the data.
calculating A inverse
  0.000049 seconds (205 allocations: 16.031 KiB)
imputing missing genotypes
  0.124649 seconds (190 allocations: 23.781 KiB, 99.87% gc time)
completed imputing genotypes
Missing values are found in independent variables: dam.

Pi (Π) is not provided.
Pi (Π) is generated assuming all markers have effects on all traits.

The prior for marker effects covariance matrix is calculated from genetic covariance matrix 

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:01




The version of Julia and Platform in use:

Julia Version 1.3.1
Commit 2d5741174c (2019-12-30 21:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


The analysis has finished. Results are saved in the returned variable and text files. MCMC samples are saved in text files.




<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [67]:
keys(out2)

Base.KeySet for a Dict{Any,Any} with 9 entries. Keys:
  "marker effects"
  "EBV_y2"
  "EBV_y1"
  "Pi"
  "location parameters"
  "residual variance"
  "polygenic effects covariance matrix"
  "EBV_y3"
  "marker effects variance"

In [68]:
out1["location parameters"]

,Trait,Effect,Level,Estimate,Std_Error
,Any,Any,Any,Any,Any
1,y1,intercept,intercept,-3.57266,2.6059
2,y1,x1*x3,x1 * m,-4.19906,6.89503
3,y1,x1*x3,x1 * f,0.479728,0.560252
4,y1,x2,2,-0.0529523,0.916848
5,y1,x2,1,-0.0712717,0.674382
6,y1,x3,m,5.86229,3.02209
7,y1,x3,f,2.34139,3.67728
8,y1,ID,a2,0.06083,0.990601
9,y1,ID,a1,0.137081,0.945244
